In [1]:
# set working directory
import os
os.chdir("..")
print(os.getcwd())

/Users/jelle/Documents/Personal/Coding Projects/Research-proj-2-TNO


In [ ]:
import pandas as pd

# Load the datasets
df = pd.read_csv("./Data/docs/4.papyrus_present_genes&compounds.csv")
pdb_df = pd.read_csv("./Data/docs/3.pdbligands&genes_combined.csv")

# Classify based on pchembl_value_Mean
df['Active'] = df['pchembl_value_Mean'] > 6.5
df['Inactive'] = df['pchembl_value_Mean'] <= 6.5

# Count the active, inactive, and datapoints (formerly Total) for each Symbol
summary = df.groupby('Symbol').agg(
    Datapoints=('SMILES', 'count'),
    Active=('Active', 'sum'),
    Inactive=('Inactive', 'sum'),
    pchembl_min=('pchembl_value_Mean', 'min'),
    pchembl_max=('pchembl_value_Mean', 'max')
).reset_index()

# Round the min and max values to 3 decimals
summary['pchembl_min'] = summary['pchembl_min'].round(3)
summary['pchembl_max'] = summary['pchembl_max'].round(3)

# Add the Ratio column (Active / Datapoints)
summary['Ratio'] = summary['Active'] / summary['Datapoints']
summary['Ratio'] = summary['Ratio'].round(3)

# Create the Range column with the min and max values for pchembl_value_Mean
summary['Activity range'] = summary['pchembl_min'].astype(str) + ' to ' + summary['pchembl_max'].astype(str)

# Count the number of PDB ligands per Symbol
pdb_counts = pdb_df.groupby('Symbol')['pdb_ligand_ID'].nunique().reset_index()
pdb_counts.rename(columns={'pdb_ligand_ID': 'PDB_ligands'}, inplace=True)

# Merge with the summary table
summary = summary.merge(pdb_counts, on='Symbol', how='left')
summary['PDB_ligands'] = summary['PDB_ligands'].fillna(0).astype(int)  # Fill missing values with 0

# Sort the table by Datapoints in descending order
summary = summary.sort_values(by='Datapoints', ascending=False)

# Show the summary table with the counts, ratio, range, and PDB ligands
summary

,Symbol,Datapoints,Active,Inactive,pchembl_min,pchembl_max,Ratio,Activity range,PDB_ligands
166,EGFR,11072,6011,5061,3.036,11.000,0.543,3.036 to 11.0,163
326,MAPK14,9344,5708,3636,3.020,13.939,0.611,3.02 to 13.939,194
302,KDR,6787,3759,3028,4.000,10.700,0.554,4.0 to 10.7,41
422,PIK3CD,6463,4837,1626,4.000,11.585,0.748,4.0 to 11.585,17
160,DRD2,5125,3114,2011,3.740,10.830,0.608,3.74 to 10.83,6
...,...,...,...,...,...,...,...,...,...
361,MUC1,1,0,1,4.120,4.120,0.000,4.12 to 4.12,8
364,MYD88,1,0,1,4.460,4.460,0.000,4.46 to 4.46,0
365,MYH2,1,0,1,4.550,4.550,0.000,4.55 to 4.55,0
366,MYOF,1,1,0,7.030,7.030,1.000,7.03 to 7.03,0


In [3]:
# Filter symbols with less than 30 actives or 30 inactives
symbols_with_less_than_30 = summary[(summary['Active'] < 30) & (summary['Inactive'] < 30)]

symbols_list_less = symbols_with_less_than_30['Symbol'].tolist()

# Print the number of such symbols
print(f"Number of targets with less than 30 actives or 30 inactives: {len(symbols_list_less)}")

Number of targets with less than 30 actives or 30 inactives: 217


In [4]:
# Filter symbols with more than 30 Active or more than 30 Inactive
symbols_with_more_than_30_active_or_inactive = summary[(summary['Active'] > 30) & (summary['Inactive'] > 30)]

# Get the list of Symbols
symbols_list_more = symbols_with_more_than_30_active_or_inactive['Symbol'].tolist()

# Print the number of such Symbols
print(f"Number of Symbols with more than 30 active and inactive: {len(symbols_list_more)}")

Number of Symbols with more than 30 active and inactive: 259


In [5]:
# Load the raw data
df_raw = pd.read_csv("./Data/docs/1.raw_genes.csv")

# Filter the raw data for symbols with more than 60 datapoints
filtered_df = df_raw[df_raw['Symbol'].isin(symbols_list_more)].copy()

# Count the number of PDB IDs in each row (split by commas) and store it in a new column 'PDB-ID Count'
filtered_df.loc[:, 'PDB-ID Count'] = filtered_df['PDB-ID'].apply(lambda x: len(x.split(',')) if pd.notnull(x) else 0)

# Filter to only rows where there's at least one PDB ID
filtered_with_pdb_count = filtered_df[filtered_df['PDB-ID Count'] > 0]

# Merge the datapoints and additional columns (Active, Inactive, Ratio) with the filtered data based on Symbol
final_df = pd.merge(filtered_with_pdb_count, summary[['Symbol', 'Datapoints', 'Active', 'Inactive', 'Ratio', 'Activity range', 'PDB_ligands']], on='Symbol', how='left')

# Filter for rows where PDB_ligands > 0 and order
final_df = final_df[final_df['PDB_ligands'] > 0]
final_df = final_df.sort_values(by='Datapoints', ascending=False)

# Show the resulting DataFrame
final_df

,Symbol,Uniprot,PDB-ID,PDB-ID Count,Datapoints,Active,Inactive,Ratio,Activity range,PDB_ligands
4,EGFR,P00533,"1IVO,1M14,1M17,1MOX,1NQL,1XKK,1YY9,1Z9I,2EB2,2...",324,11072,6011,5061,0.543,3.036 to 11.0,163
38,MAPK14,Q16539,"1A9U,1BL6,1BL7,1BMK,1DI9,1IAN,1KV1,1KV2,1M7Q,1...",252,9344,5708,3636,0.611,3.02 to 13.939,194
33,KDR,P35968,"1VR2,1Y6A,1Y6B,1YWN,2M59,2MET,2MEU,2OH4,2P2H,2...",54,6787,3759,3028,0.554,4.0 to 10.7,41
90,PIK3CD,O00329,"5DXU,5M6U,5T8F,5UBT,5VLR,6G6W,6OCO,6OCU,6PYR,6...",17,6463,4837,1626,0.748,4.0 to 11.585,17
150,DRD2,P14416,"5AER,6CM4,6LUQ,6VMS,7DFP,7JVR,8IRS,8TZQ,8U02",9,5125,3114,2011,0.608,3.74 to 10.83,6
...,...,...,...,...,...,...,...,...,...,...
169,RARG,P13631,"1EXA,1EXX,1FCX,1FCY,1FCZ,1FD0,2LBD,3LBD,4LBD,5...",11,89,51,38,0.573,4.66 to 9.92,5
0,ERBB3,P21860,"1M6B,2L9U,3KEX,3LMG,3P11,4LEO,4P59,4RIW,4RIX,4...",21,73,42,31,0.575,4.2 to 9.114,3
112,GSTO1,P78417,"1EEM,3LFL,3VLN,4IS0,4YQM,4YQU,4YQV,5UEH,5V3Q,5...",17,72,31,41,0.431,4.21 to 9.66,14
78,STAT6,P42226,"1OJ5,4Y5U,4Y5W,5D39,5NWM,5NWX,9BIG",7,70,36,34,0.514,4.05 to 9.15,2


In [6]:
# Load the raw data
df_raw = pd.read_csv("./Data/docs/1.raw_genes.csv")

# Filter the raw data for symbols with more than 60 datapoints
filtered_df = df_raw[df_raw['Symbol'].isin(symbols_list_less)].copy()

# Count the number of PDB IDs in each row (split by commas) and store it in a new column 'PDB-ID Count'
filtered_df.loc[:, 'PDB-ID Count'] = filtered_df['PDB-ID'].apply(lambda x: len(x.split(',')) if pd.notnull(x) else 0)

# Filter to only rows where there's at least one PDB ID
filtered_with_pdb_count = filtered_df[filtered_df['PDB-ID Count'] > 0]

# Merge the datapoints and additional columns (Active, Inactive, Ratio) with the filtered data based on Symbol
final_df_alt = pd.merge(filtered_with_pdb_count, summary[['Symbol', 'Datapoints', 'Active', 'Inactive', 'Ratio', 'Activity range', 'PDB_ligands']], on='Symbol', how='left')

# filter for more than 0 ligands and order
final_df_alt = final_df_alt[final_df_alt['PDB_ligands'] > 0]
final_df_alt = final_df_alt.sort_values(by='Datapoints', ascending=False)

# Show the resulting DataFrame
final_df_alt

,Symbol,Uniprot,PDB-ID,PDB-ID Count,Datapoints,Active,Inactive,Ratio,Activity range,PDB_ligands
153,WNT3A,P56704,"7DRT,7URD,7URE,8TZR",4,51,24,27,0.471,4.39 to 8.89,3
8,NOTCH1,P46531,"1PB5,1TOZ,1YYH,2F8X,2F8Y,2HE0,2VJ3,3ETO,3I08,3...",29,46,23,23,0.500,4.37 to 9.0,5
174,GAA,P10253,"5KZW,5KZX,5NN3,5NN4,5NN5,5NN6,5NN8,7P2Z,7P32",9,37,9,28,0.243,4.09 to 8.21,7
152,DCPS,Q96C86,"1ST0,1ST4,1XML,1XMM,3BL7,3BL9,3BLA,4QDE,4QDV,4...",11,36,27,9,0.750,4.62 to 10.7,10
157,HSF1,Q00613,"2LDU,5D5U,5D5V,5HDG,5HDN,7DCJ,7DCS,7DCT",8,35,18,17,0.514,4.75 to 8.15,1
...,...,...,...,...,...,...,...,...,...,...
129,FCGR3B,O75015,"1E4J,1E4K,1FNL,1T83,1T89,6EAQ",6,1,0,1,0.000,4.24 to 4.24,2
136,IL4,P05112,"1BBN,1BCN,1CYL,1HIJ,1HIK,1HZI,1IAR,1ITI,1ITL,1...",29,1,0,1,0.000,5.75 to 5.75,1
35,AGT,P01019,"1N9U,1N9V,2JP8,2WXW,2X0B,3CK0,3WOO,3WOR,4AA1,4...",20,1,1,0,1.000,8.015 to 8.015,2
34,NFE2L2,Q16236,"2FLU,2LZ1,3ZGC,4IFL,5WFV,6T7V,7K28,7K29,7K2A,7...",20,1,1,0,1.000,7.54 to 7.54,2


In [7]:
# Save to Excel
final_df.to_excel("./Data/docs/TOI_sufficient_data.xlsx", index=False)
final_df_alt.to_excel("./Data/docs/TOI_insufficient_data.xlsx", index=False)

print("Saved")

Saved
